<a href="https://colab.research.google.com/github/Kuvayasya/Coral_Travel_g/blob/main/KazantsevAV/parcer_part1_clear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# СТАРТ

In [51]:
# @title Импортирт библиотек
import pandas as pd
import numpy as np
import gdown
import re
import random
import time

from tabulate import tabulate

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
#@title Загрузка датасета "ТХС_15разд_названия_продуктов_16авг24.xlsx", присвоение название столбцам и клонирование основного столбца, чистим от NaN

# Прямая ссылка на скачивание файла
file_url = '/content/drive/MyDrive/Datasets/Стажировка_Чеки/Готово 82т_27sep24.xlsx'

# Скачиваем файл
# gdown.download(file_url, '100 тыс чеков.xlsx', quiet=False)

# Загружаем Excel файл в pandas DataFrame
df_part1 = pd.read_excel(file_url)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


## ФУНКЦИИ

In [53]:
#@title Функция 'split_rows_by_newline(df, column_name)' обработка переноса строки
def split_rows_by_newline(df, column_name):

    # Создаём новый DataFrame для хранения разделённых строк
    new_rows = []

    # Итерируем по каждой строке в DataFrame
    for index, row in df.iterrows():
        cell_value = row[column_name]

        # Проверяем, если значение в ячейке является строкой и содержит '\n'
        if isinstance(cell_value, str) and '\n' in cell_value:
            # Разделяем строку по символам переноса строки
            split_values = cell_value.split('\n')

            # Для каждого значения после разделения создаём новую строку
            for value in split_values:
                new_row = row.copy()
                new_row[column_name] = value
                new_rows.append(new_row)
        else:
            # Если символов переноса нет, оставляем строку как есть
            new_rows.append(row)

    # Создаём новый DataFrame из всех строк (старых и новых)
    new_df = pd.DataFrame(new_rows)

    # Вычисляем количество добавленных строк
    added_rows_count = len(new_df) - len(df)

    return new_df

In [54]:
#@title Функция 'remove_rows_with_newline(df, column_name)' удаления строк с переносом строки
def remove_rows_with_newline(df, column_name):

    # Фильтруем строки, которые не содержат перенос строки в указанном столбце
    filtered_df = df[~df[column_name].astype(str).str.contains('\n', na=False)]

    # Вычисляем количество удалённых строк
    removed_rows_count = len(df) - len(filtered_df)

    return filtered_df

In [55]:
#@title Функция 'print_random_rows(df, columns, colalignment, symbols=None, showind=False)' печать заданного количества случайных значений
def print_random_rows(df, columns, colalignment, symbols=None, showind=False):
    # Запрашиваем количество строк для вывода
    k = int(input("Введите количество для вывода: "))

    # Фильтруем DataFrame по указанным колонкам и удаляем строки с NaN
    filtered_df = df[columns].dropna()

    # Если передан список символов, фильтруем по ним
    if symbols:
        # Создаем маску для фильтрации строк, содержащих указанные символы
        mask = filtered_df[columns].apply(lambda x: x.str.contains('|'.join(symbols), na=False).any(), axis=1)
        filtered_df = filtered_df[mask]

    # Проверяем, достаточно ли строк для выборки
    if len(filtered_df) < k:
        print(f"В датафрейме недостаточно непустых строк. Доступно: {len(filtered_df)}")
        k = len(filtered_df)  # Устанавливаем k на максимальное доступное количество

    # Получаем случайные индексы
    index = random.sample(range(len(filtered_df)), k)

    # Фильтруем DataFrame по выбранным индексам
    df20 = filtered_df.iloc[index]

    # Печатаем результат
    print(tabulate(df20, headers='keys', tablefmt='pretty', showindex=showind, colalign=colalignment))

In [56]:
#@title Функция 'print_random_rows_new(df, columns, colalignment, symbols=None, showind=False)' печать заданного количества случайных значений со сложной структурой запроса
def print_random_rows_new(df, columns, colalignment, symbols=None, showind=False,
                      sort_column=None, sort_type=None, logic_filter=None, threshold=None):

    # Запрашиваем количество строк для вывода
    k = int(input("Введите количество для вывода: "))

    # Фильтруем DataFrame по указанным колонкам и удаляем строки с NaN
    filtered_df = df[columns].dropna()

    # Если передан список символов, фильтруем по ним
    if symbols:
        # Создаем маску для фильтрации строк, содержащих указанные символы
        mask = filtered_df[columns].apply(lambda x: x.str.contains('|'.join(symbols), na=False).any(), axis=1)
        filtered_df = filtered_df[mask]

    # Применяем логический фильтр, если указаны логическое условие и пороговое значение
    if logic_filter and threshold is not None and sort_column in columns:
        if logic_filter == 'more':
            filtered_df = filtered_df[filtered_df[sort_column] > threshold]
        elif logic_filter == 'moreandequal':
            filtered_df = filtered_df[filtered_df[sort_column] >= threshold]
        elif logic_filter == 'less':
            filtered_df = filtered_df[filtered_df[sort_column] < threshold]
        elif logic_filter == 'lessandequal':
            filtered_df = filtered_df[filtered_df[sort_column] <= threshold]

    # Проверяем, достаточно ли строк для выборки
    if len(filtered_df) < k:
        print(f"В датафрейме недостаточно непустых строк. Доступно: {len(filtered_df)}")
        k = len(filtered_df)  # Устанавливаем k на максимальное доступное количество

    # Получаем случайные индексы
    index = random.sample(range(len(filtered_df)), k)

    # Фильтруем DataFrame по выбранным индексам
    df20 = filtered_df.iloc[index]

    # Применяем сортировку по столбцу, если указана
    if sort_column and sort_column in columns:
        if sort_type == 'ltos':  # Сортировка от большего к меньшему
            df20 = df20.sort_values(by=sort_column, ascending=False)
        elif sort_type == 'stol':  # Сортировка от меньшего к большему
            df20 = df20.sort_values(by=sort_column, ascending=True)

    # Печатаем результат
    print(tabulate(df20, headers='keys', tablefmt='pretty', showindex=showind, colalign=colalignment))

In [57]:
#@title Функция 'extract_value_and_type(name)' - извлечения значения и типа
def extract_value_and_type(name):
    weight_pattern = re.compile(r'(\d+[.,]?\d*)\s*(уп|кг|гр|фл|г|мл|л|порц|пак|таб|табл|mg|kg|gr|g|l|ml|шт|pieces|pcs)', re.IGNORECASE)
    match = weight_pattern.search(name)
    if match:
        return match.group(1), match.group(2)
    return None, None

In [58]:
#@title Функция 'extract_grouped_values(df1, column_name)' поиск сложных ОВХ в (Тип2)
def extract_grouped_values(df, column_name):
    # Функция для извлечения групп с '/' или '\' из строки и удаления найденных значений
    def extract_group(text):
        #if pd.isna(text):  # Проверяем на NaN
        #    return ''  # Возвращаем пустую строку
        # Регулярное выражение для поиска чисел с '/' или '\'
        matches = re.findall(r'\d+(?:[.,]?\d*)*(?:[\/\\]\d+(?:[.,]?\d*)*)+', text)
        if matches:
            # Убираем точки в конце каждого найденного элемента
            cleaned_matches = [match.rstrip('.') for match in matches]
            # Удаляем найденные значения из текста
            # for match in cleaned_matches:
            #     text = text.replace(match, '')
            # Очищаем текст от лишних пробелов
            # text = re.sub(r'\s+', ' ', text).strip()
            return ' '.join(cleaned_matches), text  # Возвращаем найденные значения и обновлённый текст
        return None, text.strip()  # Если ничего не найдено, возвращаем пустую строку и очищенный текст

    # Применяем функцию к указанному столбцу
    df[['Type_fraction', column_name]] = df.apply(
        lambda row: pd.Series(extract_group(row[column_name])),
        axis=1
    )

    return df

In [59]:
#@title Функция 'extract_kkal(df1, column_name)' выделение значений Ккал
def extract_kkal(df, column_name):
    # Функция для извлечения числа перед 'ккал' и удаления найденного значения
    def extract_number(text):
        if pd.isna(text):  # Проверяем на NaN
            return '', text  # Возвращаем пустую строку вместо NaN
        # Регулярное выражение для поиска числа перед 'ккал'
        match = re.search(r'(\d+(?:[.,]?\d*)?)\s*ккал', text, re.IGNORECASE)
        if match:
            # Извлекаем значение калорий
            kkal_value = float(match.group(1).replace(',', '.'))
            # Удаляем найденное значение калорий из текста
            # text = text.replace(match.group(0), '')
            # Очищаем текст от лишних пробелов
            # text = re.sub(r'\s+', ' ', text).strip()
            return kkal_value, text  # Возвращаем значение калорий и очищенный текст
        return None, text.strip()  # Если число не найдено, возвращаем None и очищенный текст

    # Применяем функцию к указанному столбцу
    df[['Energy_Kkal', column_name]] = df[column_name].apply(lambda x: pd.Series(extract_number(x)))

    return df

In [60]:
#@title Функция 'extract_and_convert(df, column_name, regex1, regex2, new_column_name, conversion_factor=1)'
def extract_and_convert(df, column_name, regex1, regex2, new_column_name, conversion_factor=1):

    def extract_value(text):
        if pd.isna(text):  # Проверяем на NaN
            return None, text  # Возвращаем None и текст без изменений

        # Проверяем, содержит ли текст дефис
        # if '-' in text:
        #    return None, text  # Если да, возвращаем None и текст без изменений

        # Ищем и удаляем все комбинации цифр с дефисом
        text = re.sub(r'\d+\-\d+', '', text)

        # Ищем значение по первому регулярному выражению (число перед regex1)
        # match1 = re.search(r'(\d+[\.,]?\d*)\s*\b' + regex1 + r'\b', text)
        match1 = re.search(r'(\d+[\.,]?\d*)\s*' + regex1 + r'\b', text)
        if match1 and match1.group(1) is not None:  # Проверяем, что group(1) не None
            value1 = float(match1.group(1).replace(',', '.'))
            return round(value1, 3), text  # Если найдено, сразу возвращаем его и прекращаем поиск

        # Проверяем второй вариант регулярного выражения для regex1 (число после regex1)
        # match2 = re.search(r'\b' + regex1 + r'\s*(\d+[\.,]?\d*)', text)
        match2 = re.search(r'\b' + regex1 + r'\s*(\d+[\.,]?\d*)', text)
        if match2 and match2.group(1) is not None:  # Проверяем, что group(1) не None
            value2 = float(match2.group(1).replace(',', '.'))
            return round(value2, 3), text  # Если найдено, возвращаем и не ищем regex2

        # Ищем значение по регулярному выражению regex2 (число перед regex2)
        # match3 = re.search(r'(\d+[\.,]?\d*)\s*\b' + regex2 + r'\b', text)
        match3 = re.search(r'(\d+[\.,]?\d*)\s*' + regex2 + r'\b', text)
        if match3 and match3.group(1) is not None:  # Проверяем, что group(1) не None
            value3 = float(match3.group(1).replace(',', '.'))
            return round(value3 * conversion_factor, 3), text  # Если найдено, возвращаем его

        # Проверяем второй вариант для regex2 (число после regex2)
        # match4 = re.search(r'\b' + regex2 + r'\s*(\d+[\.,]?\d*)', text)
        match4 = re.search(r'\b' + regex2 + r'\s*(\d+[\.,]?\d*)', text)
        if match4 and match4.group(1) is not None:  # Проверяем, что group(1) не None
            value4 = float(match4.group(1).replace(',', '.'))
            return round(value4 * conversion_factor, 3), text  # Если найдено, возвращаем его

        return None, text  # Если ни одно значение не найдено

    # Применяем функцию к указанному столбцу
    df[[new_column_name, column_name]] = df[column_name].apply(lambda x: pd.Series(extract_value(x)))

    return df

In [61]:
#@title Функция 'extract_percentages(df, column_name, new_column_name)'
# Функция для извлечения значений процентов из указанного столбца DataFrame
def extract_percentages(df, column_name, new_column_name):
    def extract_percentage(text):
        if pd.isna(text):  # Проверяем на NaN
            return None  # Возвращаем None, если текст пустой

        # Ищем значение процента
        match = re.search(r'(\d+[\.,]?\d*)\s*%', text)

        if match:
            return round(float(match.group(1).replace(',', '.')), 3)  # Возвращаем значение процента

        return None  # Если процент не найден, возвращаем None

    # Применяем функцию к указанному столбцу и создаём новый столбец
    df[new_column_name] = df[column_name].apply(extract_percentage)

    return df

In [62]:
#@title Функция 'print_random_rows(df)' для вывода случайных строк
def print_random_rows(df):
    # Запрашиваем количество строк для вывода
    k = int(input("Введите количество строк для вывода: "))

    # Проверяем, достаточно ли строк в DataFrame
    if k > len(df):
        print(f"В DataFrame недостаточно строк. Доступно: {len(df)}")
        k = len(df)  # Устанавливаем k на максимальное доступное количество

    # Получаем случайные строки
    random_rows = df.sample(n=k, random_state=1)  # random_state для воспроизводимости

    # Печатаем результат
    print(tabulate(random_rows, headers='keys', tablefmt='pretty'))

In [63]:
#@title Функция 'preprocess_text(text)' приведения текста
def preprocess_text(text):
    # Приведение текста к нижнему регистру
    text = text.lower()

    # Удаление веса, объема и цифр (например, "1 кг", "500 мл", "75 гр" и т.д.)
    text = re.sub(r'\b\d+(\.\d+)?\s*(pieces|литр|litr|pcs|кг|kg|гр|gr|мл|ml|л|l|г|g|шт|%)\b', ' ', text)

    # Удаление цифр и спецсимволов (например, скобки, звездочки и прочее)
    text = re.sub(r'[^a-zа-яё\s]', ' ', text)

    # Удаляем все одиночные буквы
    text = re.sub(r'\b[a-zа-яё]\b', ' ', text)

    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [64]:
# @title Функция 'join_non_empty_values(row)' для соединения значений, пропуская пустые
def join_non_empty_values(row):
    return ' '.join(str(value) for value in row if pd.notna(value) and value != '')

In [65]:
# @title Функция 'replacement_eng_to_rus(df, column_new, column_rep)' замены английских букв на русские
def replacement_eng_to_rus(df, column_new, column_rep):
    # Создаём словарь для замены
    replacement_dict = {
        'shh': 'щ', 'shch': 'щ', 'yo': 'ё', 'jo': 'ё',
        'zh': 'ж', 'kh': 'х', 'cz': 'ц', 'ts': 'ц',
        'tc': 'ц', 'ch': 'ч', 'sh': 'ш', 'eh': 'э',
        'yu': 'ю', 'ju': 'ю', 'iu': 'ю', 'ya': 'я',
        'ja': 'я', 'a': 'а', 'b': 'б', 'c': 'ц',
        'd': 'д', 'e': 'е', 'f': 'ф', 'g': 'г',
        'h': 'х', 'i': 'и', 'j': 'й', 'k': 'к',
        'l': 'л', 'm': 'м', 'n': 'н', 'o': 'о',
        'p': 'п', 'q': 'щ', 'r': 'р', 's': 'с',
        't': 'т', 'u': 'у', 'v': 'в', 'w': 'в',
        'x': 'кс', 'y': 'й', 'z': 'з'
    }

    # Заменяем буквы
    df[column_new] = df[column_rep].replace(replacement_dict, regex=True)

    return df

## КОД

In [68]:
#@title Основной код
start_time = time.time()  # Начало отсчета времени

df_part_own = df_part1.copy()  # Создаём полную копию, чтобы избежать SettingWithCopyWarning

df_part_own.columns = ['Name', 'Sub_1', 'Sub_2']  # Назначаем столбцы

df_part_own = remove_rows_with_newline(df_part_own, 'Name') # Применяем функцию к DataFrame df_part_own для столбца 'Name'

df_part_own.loc[:, 'Name_1'] = df_part_own['Name'].str.lower() # Приводим текст к нижнему регистру

df_part_own.loc[:, 'Name'] = df_part_own['Name'].str.replace('=', ' ', regex=False) # Удаление символов "="

df_part_own['Name_1'] = df_part_own['Name_1'].str.strip()  # Убираем пробелы в начале и конце

df_part_own['Name_1'] = df_part_own['Name_1'].str.replace(r'\s+', ' ', regex=True)  # Заменяем множественные пробелы на одинарный

df_part_own = df_part_own.dropna(axis=1, how='all') # Удаляем пустые столбцы (где все значения NaN)

df_part_own = df_part_own.dropna(how='all') # Удаляем строки с пропущенными значениями

df_part_own['count'] = df_part_own['Name'].apply(len) # Добавляем новый столбец 'count' с количеством символов в столбце 'Name'

df_part_own['Значение'], df_part_own['Тип'] = zip(*df_part_own['Name_1'].apply(extract_value_and_type)) # Заполнение столбцов 'Значение' и 'Тип'

df_part_own = extract_grouped_values(df_part_own, 'Name_1') # Заполнение столбца 'Type_fraction'

df_part_own = extract_kkal(df_part_own, 'Name_1') # Заполнение столбца 'Energy, Kkal'

# Заполнение столбца 'Weight, grams'
regex1 = r'гр|gr|г|g'
regex2 = r'кг|kg'
column = 'Name_1'
column_out = 'Weight_grams'
conversion_factor = 1000
extract_and_convert(df_part_own, column, regex1, regex2, column_out, conversion_factor) # Вызываем функцию 'extract_and_convert(df, column_name, regex1, regex2, new_column_name, conversion_factor)'

# Заполнение столбца 'Volume, milliliters'
regex1 = r'мл|ml'
regex2 = r'литр|л|litr|l'
column = 'Name_1'
column_out = 'Volume_milliliters'
conversion_factor = 1000
extract_and_convert(df_part_own, column, regex1, regex2, column_out, conversion_factor) # Вызываем функцию 'extract_and_convert(df, column_name, regex1, regex2, new_column_name, conversion_factor)'

# Заполнение столбца 'Percentages, %'
column = 'Name_1'
column_out = 'Percentages'
extract_percentages(df_part_own, column, column_out) # Вызываем функцию 'extract_percentages(df_part_own, column, column_out)'

# Заполнение столбца 'Quantity, pcs'
regex1 = r'шт'
regex2 = r'pieces|pcs'
column = 'Name_1'
column_out = 'Quantity, pcs'
conversion_factor = 1
extract_and_convert(df_part_own, column, regex1, regex2, column_out, conversion_factor) # Вызываем функцию 'extract_and_convert(df, column_name, regex1, regex2, new_column_name, conversion_factor)'

df_part_own['Name_1'] = df_part_own['Name_1'].apply(preprocess_text) # Применение функции ко всем строкам столбца 'Name_old'

df_part_own = df_part_own.dropna(subset=['Name_1']) # Удаляем строки с пустыми значениями в столбце Name_1

# Создаем новый столбец Name_2, соединяя значения из указанных столбцов через пробел
df_part_own['Name_2'] = df_part_own[['Name_1', 'Значение', 'Тип',
                                       'Type_fraction', 'Energy_Kkal',
                                       'Weight_grams', 'Volume_milliliters',
                                       'Percentages', 'Quantity, pcs']].apply(join_non_empty_values, axis=1)

df_part_own['Name_2'] = df_part_own['Name_2'].str.strip()  # Убираем пробелы в начале и конце

df_part_own['Name_2'] = df_part_own['Name_2'].str.replace(r'\s+', ' ', regex=True)  # Заменяем множественные пробелы на одинарный

df_part_own['Y'] = np.where(df_part_own['Sub_1'] == 'Несъедобное', 0, 1) # Создаем новый столбец 'Y' с использованием numpy.where

df_part_own = replacement_eng_to_rus(df_part_own, 'Name_3', 'Name_1')

# Новый порядок столбцов
new_order = [
    'Name', 'Name_1', 'Name_2', 'Name_3','count', 'Значение', 'Тип',
    'Type_fraction', 'Energy_Kkal', 'Weight_grams',
    'Volume_milliliters', 'Percentages', 'Quantity, pcs',
    'Sub_1', 'Sub_2', 'Y'
]



df_part_own = df_part_own[new_order] # Устанавливаем новый порядок столбцов
df_for_model = df_part_own[['Name', 'Name_1', 'Name_2', 'Name_3', 'Y']].copy() # Сохраняем только нужные столбцы в новом DataFrame

end_time = time.time()  # Конец отсчета времени
execution_time = end_time - start_time
print(f"Время выполнения: {execution_time:.2f} секунд")

Время выполнения: 70.12 секунд


## СОХРАНЕНИЕ

In [71]:
df_part_own.to_excel('/content/drive/MyDrive/Datasets/Стажировка_Чеки/parcing_result_end.xlsx', index=False) # Сохраняем основной очищенный и подготовленный DataFrame в Excel
df_part_own.to_excel('/content/drive/MyDrive/Datasets/Стажировка_Чеки/parcing_result_for_model.xlsx', index=False) # Сохранение подготовленный DataFrameм для обучение бинарной модели в Excel

# ПЕЧАТЬ

In [70]:
# Вызываем функцию
print_random_rows(df_part_own)
print_random_rows(df_for_model)

Введите количество строк для вывода: 20
+-------+--------------------------------------------------------------------------+------------------------------------------------------------------+------------------------------------------------------------------+------------------------------------------------------------------+-------+----------+------+---------------+-------------+--------------+--------------------+-------------+---------------+-----------------------------------------------------+------------------------------------------------------------------+---+
|       |                                   Name                                   |                              Name_1                              |                              Name_2                              |                              Name_3                              | count | Значение | Тип  | Type_fraction | Energy_Kkal | Weight_grams | Volume_milliliters | Percentages | Quantity, pcs |                    